In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

# Process
# 1. List of Docs
# 2. for doc in list of docs | prompt | llm
# 3. for response in list of llms response above | put them all together
# 4. final doc | prompt | llm

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

# 5. map_doc_prompt를 가지고 각각의 문서를 llm을 통해 처리함
map_doc_chain = map_doc_prompt | llm


# 4. map_chain의 documents와 question을 가져와서 map_doc_chain으로 각각의 문서를 질문과 함께 처리함. 그 후 각각의 결과(content)를 합침.
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    # results=[]
    # for document in documents:
    #     result =map_doc_chain.invoke({
    #         "context": document.page_content, "question": question
    #     }).content
    #     results.append(result)
    # results="\n\n".join(results)
    # return results

    # 위를 더 간결하고 보기 좋고 효율적으로 코딩할 수 있음
    # 6. 각각의 문서의 답변을 하나의 string으로 합침
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )


# 2. retriever를 이용하여 조각 문서를 가져온 후 질문과 함께 map_docs 호출
map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
    # 3. RunnableLamda는 Chain과 그 내부 어디에서든 함수를 호출할 수 있게 함
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

# 1. map_chain을 호출 후 결과가 반환되면 이를 가지고 final_prompt 호출
chain = (
    {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm
)  # 7. 합쳐진 결과를 가지고 llm을 이용해 최종 답변을 받음

chain.invoke("How many ministries are mentioned")

AIMessage(content='Three ministries are mentioned in the text: the Ministry of Love, the Ministry of Plenty, and the Ministry of Truth.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 140, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a5d6fa15-f985-4dfb-9fbf-52d6a5547b53-0')